In [1]:
import pandas as pd
import numpy as np 
import sqlite3

In [2]:
df = pd.read_csv(r"ENTRADASEBZMORUMBI2025 - Entradas.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 53 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0                                  813 non-null    object 
 1   ID                             2000 non-null   float64
 2   Colaborador                    812 non-null    object 
 3   Cliente                        812 non-null    object 
 4   Servico                        812 non-null    object 
 5   ValordoServico                 812 non-null    object 
 6   ClienteNovo                    805 non-null    object 
 7   Local                          805 non-null    object 
 8   Produto                        812 non-null    object 
 9   QuantidadeProduto              812 non-null    object 
 10  ValorTotaldosProduto           927 non-null    object 
 11  AuxValorTotalCompraProdutos    946 non-null    object 
 12  AuxDescontoProdutos            812 non-null    o

In [4]:
df = df.dropna(subset=("Colaborador"))
df.shape

(812, 53)

In [5]:
df.columns

Index([' ', 'ID', 'Colaborador', 'Cliente', 'Servico', 'ValordoServico',
       'ClienteNovo', 'Local', 'Produto', 'QuantidadeProduto',
       'ValorTotaldosProduto', 'AuxValorTotalCompraProdutos',
       'AuxDescontoProdutos', 'PorcentagemProdutoColaborador', 'Doces',
       'QuantidadeDoces', 'ValorTotaldosDoces', 'AuxValorTotalCompraDoces',
       'AuxDescontoDoces', 'Salgados', 'QuantidadeSalgados',
       'ValorTotaldosSalgados', 'AuxValorTotalComprasSalgados',
       'AuxDescontoSalgados', 'Bebidas', 'QuantidadeBebidas',
       'ValorTotaldasBebidas', 'AuxValorTotalComprasBebidas',
       'AuxDescontoBebidas', 'FormadePagamento', 'TaxaMaquina', 'Total(S+P)',
       'Total(S+P)*T', 'Total[(S+P)-LP]*T', 'TotalS+LP-Col',
       'TotalColaborador', 'Colaborador50%', 'AuxDadosProdutos',
       'AuxDatasProdutos', 'Unnamed: 39', 'DATA', 'Unnamed: 41', 'PRODUTOS',
       'AuxQuantidadeProdutos', 'AuxDescontoProdutos.1', 'AuxTaxas',
       'TaxaFinal', 'ID.1', 'Unnamed: 48', 'Unnamed: 49

In [6]:
df.isna().sum()

                                   0
ID                                 0
Colaborador                        0
Cliente                            0
Servico                            0
ValordoServico                     0
ClienteNovo                        7
Local                              7
Produto                            0
QuantidadeProduto                  0
ValorTotaldosProduto               0
AuxValorTotalCompraProdutos        0
AuxDescontoProdutos                0
PorcentagemProdutoColaborador      0
Doces                              0
QuantidadeDoces                    0
ValorTotaldosDoces                 0
AuxValorTotalCompraDoces           0
AuxDescontoDoces                   0
Salgados                           0
QuantidadeSalgados                 0
ValorTotaldosSalgados              0
AuxValorTotalComprasSalgados       0
AuxDescontoSalgados                0
Bebidas                            0
QuantidadeBebidas                  0
ValorTotaldasBebidas               0
A

In [7]:
df.drop(columns=["Unnamed: 48","Unnamed: 49"],inplace=True) 

In [8]:
df.columns

Index([' ', 'ID', 'Colaborador', 'Cliente', 'Servico', 'ValordoServico',
       'ClienteNovo', 'Local', 'Produto', 'QuantidadeProduto',
       'ValorTotaldosProduto', 'AuxValorTotalCompraProdutos',
       'AuxDescontoProdutos', 'PorcentagemProdutoColaborador', 'Doces',
       'QuantidadeDoces', 'ValorTotaldosDoces', 'AuxValorTotalCompraDoces',
       'AuxDescontoDoces', 'Salgados', 'QuantidadeSalgados',
       'ValorTotaldosSalgados', 'AuxValorTotalComprasSalgados',
       'AuxDescontoSalgados', 'Bebidas', 'QuantidadeBebidas',
       'ValorTotaldasBebidas', 'AuxValorTotalComprasBebidas',
       'AuxDescontoBebidas', 'FormadePagamento', 'TaxaMaquina', 'Total(S+P)',
       'Total(S+P)*T', 'Total[(S+P)-LP]*T', 'TotalS+LP-Col',
       'TotalColaborador', 'Colaborador50%', 'AuxDadosProdutos',
       'AuxDatasProdutos', 'Unnamed: 39', 'DATA', 'Unnamed: 41', 'PRODUTOS',
       'AuxQuantidadeProdutos', 'AuxDescontoProdutos.1', 'AuxTaxas',
       'TaxaFinal', 'ID.1', 'AuxDescontoProdutos.2', 'A

In [9]:
df.rename(columns={' ': "data"}, inplace=True)

In [10]:
class data_cleaning(pd.DataFrame):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def to_date(self, date_column):
        self[date_column] = pd.to_datetime(self[date_column], format="%d/%m/%Y")
        return self
        
    def normalize_dollar(self, dollar_column):
        if dollar_column in self.columns:
            self[dollar_column] = self[dollar_column].apply(lambda x: x.replace("R$ ","").replace(",",".").replace(" ","").replace("0.0.0","0") if isinstance(x,str) else x)
            self[dollar_column] = self[dollar_column].astype(float)
        return self
    
    def to_boolean(self, dollar_column):
        if dollar_column in self.columns:
            self[dollar_column] = self[dollar_column].map({"NÃO": False, "SIM": True}).astype(bool)
        return self   
    
    def to_null(self, dollar_column):
        if dollar_column in self.columns:
            self[dollar_column] = self[dollar_column].apply(lambda x: np.nan if x == "VAZIO" else x)
        return self
    
    def rename_second_column(self):
        if len(self.columns) > 1:
            self.rename(columns={self.columns[0]: "produto"}, inplace=True)
            self.rename(columns={self.columns[1]: "vl_venda"}, inplace=True)
            self.rename(columns={self.columns[2]: "compra"}, inplace=True)
            self.rename(columns={self.columns[3]: "lucro"}, inplace=True)
        return self
    
class feature_engineering(data_cleaning):
     
     def __init__(self,*args,**kwargs):
         super().__init__(*args,**kwargs)
         
     def index_values(self,column):
         unique_values = self[column].unique()
         dicionario = {value: key for key, value in enumerate(unique_values)}
         self[f"id_{column}"] = self[column].map(dicionario)
         return self
    

In [11]:
new_df = data_cleaning(df)
new_df = new_df.to_date("data")
new_df.info()

<class '__main__.data_cleaning'>
Index: 812 entries, 0 to 811
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   data                           812 non-null    datetime64[ns]
 1   ID                             812 non-null    float64       
 2   Colaborador                    812 non-null    object        
 3   Cliente                        812 non-null    object        
 4   Servico                        812 non-null    object        
 5   ValordoServico                 812 non-null    object        
 6   ClienteNovo                    805 non-null    object        
 7   Local                          805 non-null    object        
 8   Produto                        812 non-null    object        
 9   QuantidadeProduto              812 non-null    object        
 10  ValorTotaldosProduto           812 non-null    object        
 11  AuxValorTotalCompraProdutos  

In [12]:
colunas_valor = [col for col in df.columns if col.startswith("Valor")] + ["TaxaMaquina","Total(S+P)","Total(S+P)*T","Total[(S+P)-LP]*T","TotalS+LP-Col","TotalColaborador","Colaborador50%"] + [col for col in df.columns if col.startswith("AuxValor")] + [col for col in df.columns if col.startswith("AuxDesconto")]
for col in colunas_valor:
    new_df = new_df.normalize_dollar(col)
new_df.info()


<class '__main__.data_cleaning'>
Index: 812 entries, 0 to 811
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   data                           812 non-null    datetime64[ns]
 1   ID                             812 non-null    float64       
 2   Colaborador                    812 non-null    object        
 3   Cliente                        812 non-null    object        
 4   Servico                        812 non-null    object        
 5   ValordoServico                 812 non-null    float64       
 6   ClienteNovo                    805 non-null    object        
 7   Local                          805 non-null    object        
 8   Produto                        812 non-null    object        
 9   QuantidadeProduto              812 non-null    object        
 10  ValorTotaldosProduto           812 non-null    float64       
 11  AuxValorTotalCompraProdutos  

In [13]:
colunas_valor = [col for col in df.columns if col.startswith("AuxValor")]
for col in colunas_valor:
    new_df = new_df.normalize_dollar(col)
new_df.info()

<class '__main__.data_cleaning'>
Index: 812 entries, 0 to 811
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   data                           812 non-null    datetime64[ns]
 1   ID                             812 non-null    float64       
 2   Colaborador                    812 non-null    object        
 3   Cliente                        812 non-null    object        
 4   Servico                        812 non-null    object        
 5   ValordoServico                 812 non-null    float64       
 6   ClienteNovo                    805 non-null    object        
 7   Local                          805 non-null    object        
 8   Produto                        812 non-null    object        
 9   QuantidadeProduto              812 non-null    object        
 10  ValorTotaldosProduto           812 non-null    float64       
 11  AuxValorTotalCompraProdutos  

In [14]:
colunas_valor = [col for col in df.columns if col.startswith("AuxDesconto")]
for col in colunas_valor:
    new_df = new_df.normalize_dollar(col)
new_df.info()

<class '__main__.data_cleaning'>
Index: 812 entries, 0 to 811
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   data                           812 non-null    datetime64[ns]
 1   ID                             812 non-null    float64       
 2   Colaborador                    812 non-null    object        
 3   Cliente                        812 non-null    object        
 4   Servico                        812 non-null    object        
 5   ValordoServico                 812 non-null    float64       
 6   ClienteNovo                    805 non-null    object        
 7   Local                          805 non-null    object        
 8   Produto                        812 non-null    object        
 9   QuantidadeProduto              812 non-null    object        
 10  ValorTotaldosProduto           812 non-null    float64       
 11  AuxValorTotalCompraProdutos  

In [15]:
new_df.head()

,data,ID,Colaborador,Cliente,Servico,ValordoServico,ClienteNovo,Local,Produto,QuantidadeProduto,...,Unnamed: 41,PRODUTOS,AuxQuantidadeProdutos,AuxDescontoProdutos.1,AuxTaxas,TaxaFinal,ID.1,AuxDescontoProdutos.2,AuxTaxas.1,TaxaFinal.1
0,2025-01-07,1.0,JORDI,João Paulo Carvalho Souza,Corte+Barba+Sobrancelha,110.0,NÃO,FIDELIZADO,VAZIO,0,...,Yopro Baunilha 250 mL,Yopro Baunilha 250 mL,NaN,NaN,"0,985",NaN,FAZER DEPOIS,NaN,"0,99",NaN
1,2025-01-07,2.0,JORDI,Rafinha,Corte,60.0,SIM,INDICAÇÃO,VAZIO,0,...,Suco Natural One Uva 180 mL,Suco Natural One Uva 180 mL,NaN,NaN,"0,985",NaN,DEIXAR PARA,NaN,"0,985",NaN
2,2025-01-07,3.0,JORDI,Luquinha,Corte,60.0,SIM,INDICAÇÃO,VAZIO,0,...,BROWNIE DOCE DE LEITE QUERO+,BROWNIE DOCE DE LEITE QUERO+,NaN,NaN,"0,99",NaN,DEPOIS QUE,NaN,"0,99",NaN
3,2025-01-07,4.0,JORDI,Adriano Duarte,Corte,60.0,SIM,INDICAÇÃO,VAZIO,0,...,Água Crystal Com Gás 500 mL,Água Crystal Com Gás 500 mL,NaN,NaN,"0,9725",NaN,TERMINAR O,NaN,"0,9725",NaN
4,2025-01-07,5.0,JORDI,Diego,Corte+Barba,90.0,SIM,FIDELIZADO,VAZIO,0,...,Balm Midtown B.URB,Balm Midtown B.URB,NaN,NaN,"0,9725",NaN,RESTO DA,NaN,"0,9725",NaN


In [16]:
new_df = new_df.to_boolean("ClienteNovo")
new_df.info()

<class '__main__.data_cleaning'>
Index: 812 entries, 0 to 811
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   data                           812 non-null    datetime64[ns]
 1   ID                             812 non-null    float64       
 2   Colaborador                    812 non-null    object        
 3   Cliente                        812 non-null    object        
 4   Servico                        812 non-null    object        
 5   ValordoServico                 812 non-null    float64       
 6   ClienteNovo                    812 non-null    bool          
 7   Local                          805 non-null    object        
 8   Produto                        812 non-null    object        
 9   QuantidadeProduto              812 non-null    object        
 10  ValorTotaldosProduto           812 non-null    float64       
 11  AuxValorTotalCompraProdutos  

In [18]:
colunas = ["Produto","Doces","Salgados","Bebidas"]
for col in colunas:
    new_df = new_df.to_null(col)
new_df.head()

,data,ID,Colaborador,Cliente,Servico,ValordoServico,ClienteNovo,Local,Produto,QuantidadeProduto,...,Unnamed: 41,PRODUTOS,AuxQuantidadeProdutos,AuxDescontoProdutos.1,AuxTaxas,TaxaFinal,ID.1,AuxDescontoProdutos.2,AuxTaxas.1,TaxaFinal.1
0,2025-01-07,1.0,JORDI,João Paulo Carvalho Souza,Corte+Barba+Sobrancelha,110.0,False,FIDELIZADO,NaN,0,...,Yopro Baunilha 250 mL,Yopro Baunilha 250 mL,NaN,NaN,"0,985",NaN,FAZER DEPOIS,NaN,"0,99",NaN
1,2025-01-07,2.0,JORDI,Rafinha,Corte,60.0,True,INDICAÇÃO,NaN,0,...,Suco Natural One Uva 180 mL,Suco Natural One Uva 180 mL,NaN,NaN,"0,985",NaN,DEIXAR PARA,NaN,"0,985",NaN
2,2025-01-07,3.0,JORDI,Luquinha,Corte,60.0,True,INDICAÇÃO,NaN,0,...,BROWNIE DOCE DE LEITE QUERO+,BROWNIE DOCE DE LEITE QUERO+,NaN,NaN,"0,99",NaN,DEPOIS QUE,NaN,"0,99",NaN
3,2025-01-07,4.0,JORDI,Adriano Duarte,Corte,60.0,True,INDICAÇÃO,NaN,0,...,Água Crystal Com Gás 500 mL,Água Crystal Com Gás 500 mL,NaN,NaN,"0,9725",NaN,TERMINAR O,NaN,"0,9725",NaN
4,2025-01-07,5.0,JORDI,Diego,Corte+Barba,90.0,True,FIDELIZADO,NaN,0,...,Balm Midtown B.URB,Balm Midtown B.URB,NaN,NaN,"0,9725",NaN,RESTO DA,NaN,"0,9725",NaN


In [19]:
new_df.isnull().sum()

data                               0
ID                                 0
Colaborador                        0
Cliente                            0
Servico                            0
ValordoServico                     0
ClienteNovo                        0
Local                              7
Produto                          791
QuantidadeProduto                  0
ValorTotaldosProduto               0
AuxValorTotalCompraProdutos        0
AuxDescontoProdutos                0
PorcentagemProdutoColaborador      0
Doces                            781
QuantidadeDoces                    0
ValorTotaldosDoces                 0
AuxValorTotalCompraDoces           0
AuxDescontoDoces                   0
Salgados                         796
QuantidadeSalgados                 0
ValorTotaldosSalgados              0
AuxValorTotalComprasSalgados       0
AuxDescontoSalgados                0
Bebidas                          772
QuantidadeBebidas                  0
ValorTotaldasBebidas               0
A

In [20]:
print(new_df.shape)
new_df = new_df.assign(Bebidas=new_df["Bebidas"].str.split(",")).explode("Bebidas")
print(new_df.shape)

(812, 51)
(817, 51)


In [21]:
new_df["ID"].nunique()/new_df["Cliente"].nunique()

2.1256544502617802

In [22]:
produtos = pd.read_csv(r"c:\Users\luiz\Documents\GitHub\EZmorumbi\Data Cleaning\ENTRADASEBZMORUMBI2025 - Produtos.csv")


In [23]:
dim_servico = produtos[["SERVIÇO","VALOR DO SERVIÇO"]]
dim_produto = produtos[["PRODUTO","VALOR DE VENDA","VALOR DE COMPRA","LUCRO"]]
dim_doce = produtos[["DOCE","VALOR DE VENDA.1","VALOR DE COMPRA.1","LUCRO.1"]]
dim_salgado = produtos[["SALGADO","VALOR DE VENDA.2","VALOR DE COMPRA.2","LUCRO.2"]]
dim_bebida = produtos[["BEBIDA","VALOR DE VENDA.3","VALOR DE COMPRA.3","LUCRO.3"]]
dim_funcionario = produtos[["FUNCIONARIO","PORCENTAGEM"]]
dim_taxa = produtos[["FUNCAO","TAXA"]]


In [24]:
dim_servico["tipo"] = "servico"
dim_doce["tipo"] = "doce"
dim_salgado["tipo"] = "salgado"
dim_bebida["tipo"] = "bebida"
dim_funcionario["tipo"] = "funcionario"
dim_taxa["tipo"] = "taxa"

C:\Users\luiz\AppData\Local\Temp\ipykernel_14740\61306986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_servico["tipo"] = "servico"
C:\Users\luiz\AppData\Local\Temp\ipykernel_14740\61306986.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_doce["tipo"] = "doce"
C:\Users\luiz\AppData\Local\Temp\ipykernel_14740\61306986.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [25]:
dataframes = [dim_servico,dim_produto,dim_doce,dim_salgado,dim_bebida,dim_funcionario,dim_taxa]

dim_servico = data_cleaning(dim_servico)
dim_produto = data_cleaning(dim_produto)
dim_doce = data_cleaning(dim_doce)
dim_salgado = data_cleaning(dim_salgado)
dim_bebida = data_cleaning(dim_bebida)

In [26]:
dim_produto.head()

,PRODUTO,VALOR DE VENDA,VALOR DE COMPRA,LUCRO
0,Balm Midtown B.URB,"R$ 65,00","R$ 37,00","R$ 28,00"
1,Óleo para Barba Venice B.URB,"R$ 65,00","R$ 45,00","R$ 20,00"
2,Pomada Queens B.URB,"R$ 75,00","R$ 46,00","R$ 29,00"
3,Pomada Brooklyn B.URB,"R$ 75,00","R$ 51,00","R$ 24,00"
4,Minoxidil Kirkland,"R$ 110,00","R$ 55,00","R$ 55,00"


In [27]:
dim_produto.rename_second_column()
dim_doce.rename_second_column()
dim_salgado.rename_second_column()
dim_bebida.rename_second_column()

,produto,vl_venda,compra,lucro,tipo
0,Água Crystal Com Gás 500 mL,"R$ 5,00","R$ 1,89","R$ 3,11",bebida
1,Red Bull 250 mL,"R$ 15,00","R$ 7,99","R$ 7,01",bebida
2,Água Crystal Sem Gás 500 mL,"R$ 5,00","R$ 1,65","R$ 3,35",bebida
3,Fanta Laranja 350 mL,"R$ 7,00","R$ 2,99","R$ 4,01",bebida
4,Refrigerante Sprite 350 mL,"R$ 7,00","R$ 2,99","R$ 4,01",bebida
...,...,...,...,...,...
108,NaN,NaN,NaN,NaN,bebida
109,NaN,NaN,NaN,NaN,bebida
110,NaN,NaN,NaN,NaN,bebida
111,NaN,NaN,NaN,NaN,bebida


In [28]:
for data in dataframes:
    data.dropna(inplace=True)

C:\Users\luiz\AppData\Local\Temp\ipykernel_14740\3987279053.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)
C:\Users\luiz\AppData\Local\Temp\ipykernel_14740\3987279053.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [29]:
dim_servico.rename(columns={
    "SERVIÇO" : "servico",
    "VALOR DO SERVIÇO" : "vl_venda"
})

,servico,vl_venda,tipo
0,Corte,"R$ 60,00",servico
1,Barba,"R$ 50,00",servico
2,Perfil,"R$ 20,00",servico
3,Sobrancelha,"R$ 20,00",servico
4,Higienização Facial,"R$ 50,00",servico
...,...,...,...
108,Corte+Barba+Sobrancelha+Luzes,"R$ 260,00",servico
109,Corte+Sobrancelha+Luzes,"R$ 230,00",servico
110,Corte+Barba+Sobrancelha+Platinado,"R$ 360,00",servico
111,Corte+Sobrancelha+Platinado,"R$ 320,00",servico


In [30]:
dim_produto = dim_produto.normalize_dollar("lucro")
dim_doce = dim_doce.normalize_dollar("lucro")
dim_salgado = dim_salgado.normalize_dollar("lucro")
dim_bebida = dim_bebida.normalize_dollar("lucro")
dim_produto = dim_produto.normalize_dollar("vl_venda")
dim_doce = dim_doce.normalize_dollar("vl_venda")
dim_salgado = dim_salgado.normalize_dollar("vl_venda")
dim_bebida = dim_bebida.normalize_dollar("vl_venda")
dim_produto = dim_produto.normalize_dollar("compra")
dim_doce = dim_doce.normalize_dollar("compra")
dim_salgado = dim_salgado.normalize_dollar("compra")
dim_bebida = dim_bebida.normalize_dollar("compra")

In [31]:
tabela_fato = new_df[['data', 'ID', 'Colaborador', 'Cliente', 'Servico', 'ValordoServico',
       'ClienteNovo', 'Local', 'Produto', 'QuantidadeProduto',
       'ValorTotaldosProduto', 'Doces',
       'QuantidadeDoces', 'ValorTotaldosDoces', 'Salgados', 'QuantidadeSalgados',
       'Bebidas', 'QuantidadeBebidas',
       'ValorTotaldasBebidas', 'FormadePagamento', 'TaxaMaquina', 'Total(S+P)',
       'Total(S+P)*T', 'Total[(S+P)-LP]*T', 'TotalS+LP-Col',
       'TotalColaborador', 'Colaborador50%']]

In [32]:
tabela_fato = tabela_fato.rename(columns={
    'data': 'data',
    'ID': 'id',
    'Colaborador': 'colaborador',
    'Cliente': 'cliente',
    'Servico': 'servico',
    'ValordoServico': 'vl_servico',
    'ClienteNovo': 'cliente_novo',
    'Local': 'local',  
    'Produto': 'produto',
    'QuantidadeProduto': 'qtd_produto',
    'ValorTotaldosProduto': 'vl_total_produto',
    'Doces': 'doce',
    'QuantidadeDoces': 'qtd_doce',
    'ValorTotaldosDoces': 'vl_total_doces',
    'Salgados': 'salgado',  
    'QuantidadeSalgados': 'qtd_salgado',
    'ValorTotaldosSalgados': 'vl_total_salgado',
    'Bebidas': 'bebida',  
    'QuantidadeBebidas': 'qtd_bebida',
    'ValorTotaldasBebidas': 'vl_total_bebida',
    'FormadePagamento': 'forma_de_pagamento',
    'TaxaMaquina': 'tx_maquina',
    'Total(S+P)': 'total_sp',
    'Total(S+P)*T': 'total_spt',
    'Total[(S+P)-LP]*T': 'total_splt',
    'TotalS+LP-Col': 'total_slp_col',
    'TotalColaborador': 'total_colaborador',
    'Colaborador50%': 'colaborador_50'
})

In [33]:
tabela_fato = tabela_fato.rename(columns={
    'local': 'fidelizado'
})

In [34]:
dim_produto.head()

,produto,vl_venda,compra,lucro
0,Balm Midtown B.URB,65.0,37.0,28.0
1,Óleo para Barba Venice B.URB,65.0,45.0,20.0
2,Pomada Queens B.URB,75.0,46.0,29.0
3,Pomada Brooklyn B.URB,75.0,51.0,24.0
4,Minoxidil Kirkland,110.0,55.0,55.0


In [35]:
def to_string(dataframe,coluna):
    dataframe[coluna] = dataframe[coluna].astype(str)
    return dataframe[coluna].apply(lambda x: type(x)).value_counts()

In [36]:
to_string(dim_bebida,"produto")

produto
<class 'str'>    113
Name: count, dtype: int64

In [37]:
to_string(dim_produto,"produto")

produto
<class 'str'>    113
Name: count, dtype: int64

In [38]:
dim_produto = dim_produto[dim_produto["produto"] != "VAZIO"]

In [39]:
dataframes = [dim_bebida, dim_doce, dim_salgado]
for df in dataframes:
    df.dropna(subset=["vl_venda"],inplace=True)

In [40]:
dim_produto_2 = pd.concat([dim_bebida, dim_doce, dim_salgado])

In [41]:
dim_produto_2.head(100)

,produto,vl_venda,compra,lucro,tipo
0,Água Crystal Com Gás 500 mL,5.0,1.89,3.11,bebida
1,Red Bull 250 mL,15.0,7.99,7.01,bebida
2,Água Crystal Sem Gás 500 mL,5.0,1.65,3.35,bebida
3,Fanta Laranja 350 mL,7.0,2.99,4.01,bebida
4,Refrigerante Sprite 350 mL,7.0,2.99,4.01,bebida
5,Yopro Café Expresso 250 mL,12.0,5.99,6.01,bebida
6,Yopro Baunilha 250 mL,12.0,5.99,6.01,bebida
7,Yopro Chocolate 250 mL,12.0,5.99,6.01,bebida
8,Yopro Morango 250 mL,12.0,5.99,6.01,bebida
9,Yopro Banana 250 mL,12.0,5.99,6.01,bebida


In [42]:
dim_p_b = dim_produto.merge(dim_produto_2, on="produto", how="left", suffixes=("", "n_"))
dim_p_b.drop(columns=["vl_vendan_","compran_","lucron_"],inplace=True)

In [43]:
dim_p_b.dropna(subset="vl_venda",inplace=True)

In [44]:
dim_p_b["tipo"] = dim_p_b["tipo"].apply(lambda x: "barbearia" if pd.isna(x) else x)
dim_p_b.head()

,produto,vl_venda,compra,lucro,tipo
0,Balm Midtown B.URB,65.0,37.0,28.0,barbearia
1,Óleo para Barba Venice B.URB,65.0,45.0,20.0,barbearia
2,Pomada Queens B.URB,75.0,46.0,29.0,barbearia
3,Pomada Brooklyn B.URB,75.0,51.0,24.0,barbearia
4,Minoxidil Kirkland,110.0,55.0,55.0,barbearia


In [45]:
dim_p_b.drop_duplicates(subset=["produto"],inplace=True)
dim_p_b = feature_engineering(dim_p_b)
dim_p_b = dim_p_b.index_values("produto")
dim_p_b.head()

,produto,vl_venda,compra,lucro,tipo,id_produto
0,Balm Midtown B.URB,65.0,37.0,28.0,barbearia,0
1,Óleo para Barba Venice B.URB,65.0,45.0,20.0,barbearia,1
2,Pomada Queens B.URB,75.0,46.0,29.0,barbearia,2
3,Pomada Brooklyn B.URB,75.0,51.0,24.0,barbearia,3
4,Minoxidil Kirkland,110.0,55.0,55.0,barbearia,4


In [48]:
produto_map = dim_p_b.set_index("produto")["id_produto"].to_dict()

In [49]:
produto_map

{'Balm Midtown B.URB': 0,
 'Óleo para Barba Venice B.URB': 1,
 'Pomada Queens B.URB': 2,
 'Pomada Brooklyn B.URB': 3,
 'Minoxidil Kirkland': 4,
 'PENTE MASSAGEADOR': 5,
 'Máscara Black Red Nek': 6,
 'Esfoliante Facial B.URB': 7,
 'Kit de Higienização Facial': 8,
 'Leave-in B.URB': 9,
 'Toppik': 10,
 'Laque Charming 400 mL': 11,
 'GEL COLA CRISTAL RED NEK': 12,
 'GELATINA PARA CACHOS INFINITY': 13,
 'Máscara Hidratante de Coco Infinity': 14,
 'Kit para Cabelo B.URB': 15,
 'Kit para Barba B.URB': 16,
 'Shampoo B.URB 250 mL': 17,
 'Condicionador B.URB 250 mL': 18,
 'PÃO DE MEL BRIGADEIRO URITS': 19,
 'PÃO DE MEL DOCE DE LEITE URITS': 20,
 'PALHA ITALIANA TRADICIONAL URITS': 21,
 'WAFFE URITS': 22,
 'NOODLE NUTS URITS': 23,
 'BROWNIE CHOCOWAFER QUERO+': 24,
 'BROWNIE DOCE DE LEITE QUERO+': 25,
 'BROWNIE CREME DE AVELA QUERO+': 26,
 'BROWNIE BRIGADEIRO QUERO+': 27,
 'BROWNIE CHOCOLATE MALTADO QUERO+': 28,
 'BROWNIE LEITE QUERO+': 29,
 'AMENDOIM JAPONES ELMA CHIPS 145 G': 30,
 'AMENDOIM SEM 

In [ ]:
tabela_fato["produto"] = tabela_fato["produto"].map(produto_map)
tabela_fato["doce"] = tabela_fato["doce"].map(produto_map)
tabela_fato["salgado"] = tabela_fato["salgado"].map(produto_map)
tabela_fato["bebida"] = tabela_fato["bebida"].map(produto_map)

In [ ]:
tabela_fato.head(10)

In [ ]:
df.info()

In [ ]:
tabela_fato.info()

In [ ]:
import mysql.connector
import pandas as pd

def create_table(cursor):
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS tabela_fato (
            data DATETIME,
            id DOUBLE,
            colaborador TEXT,
            cliente TEXT,
            servico TEXT,
            vl_servico DOUBLE,
            cliente_novo BOOLEAN,
            fidelizado TEXT,
            produto TEXT,
            qtd_produto TEXT,
            vl_total_produto DOUBLE,
            doce TEXT,
            qtd_doce DOUBLE,
            vl_total_doces DOUBLE,
            salgado TEXT,
            qtd_salgado DOUBLE,
            bebida TEXT,
            qtd_bebida TEXT,
            vl_total_bebida DOUBLE,
            forma_de_pagamento TEXT,
            tx_maquina DOUBLE,
            total_sp DOUBLE,
            total_spt DOUBLE,
            total_splt DOUBLE,
            total_slp_col DOUBLE,
            total_colaborador DOUBLE,
            colaborador_50 DOUBLE
        )
    ''')

def insert_data(cursor, df):
    for _, row in df.iterrows():
        cursor.execute('''
            INSERT INTO tabela_fato VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', tuple(row))

def main(df):
    conn = mysql.connector.connect(
        host='database-atinova.ct6oomqu6y49.sa-east-1.rds.amazonaws.com',
        user='integrantes',
        password='grupoPI2025',
        database='barbearia',  # Substituir pelo nome correto do banco de dados
        port=3306
    )
    cursor = conn.cursor()
    
    create_table(cursor)
    
    df.fillna('', inplace=True)  # Substituir NaN por strings vazias
    insert_data(cursor, df)
    
    conn.commit()
    cursor.close()
    conn.close()

# Chame a função passando seu DataFrame diretamente
main(tabela_fato)
